In [3]:
import os
data_path = os.path.expanduser('C:\\Users\\might\\Desktop\\jupyter notebooks\\u.data')
df= pd.read_csv(data_path, sep='\t',names= 'user_id,movie_id,rating,timestamp'.split(','))#, header=None)#used for DeepCTR

sparse_features = ["movie_id", "user_id"]
y= ['rating']
df.head(5)

,user_id,movie_id,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


* To be used in surpriselib
* above `trainset` & `testset` folds will be consistent across surprise SVD, NN_SVD, DeepCTR's SVD

In [4]:
from sklearn.model_selection import train_test_split
trainset, testset = train_test_split(df, test_size=0.2)

W1004 19:52:48.781906 23276 utils.py:41] 
DeepCTR version 0.6.2 detected. Your version is 0.6.1.
Use `pip install -U deepctr` to upgrade.Changelog: https://github.com/shenweichen/DeepCTR/releases/tag/v0.6.2


In [60]:
??Dataset.load_from_df

In [64]:
from surprise import SVD
from surprise import Dataset
from surprise import Reader

rdr = Reader(rating_scale=(1,5))
data= Dataset.load_from_df(trainset[sparse_features+ y], reader= rdr)

In [79]:
train_s = data.build_full_trainset()
from surprise import accuracy
import time

t1= time.time()

algo_svd = SVD(n_factors=100, n_epochs=128)

algo_svd.fit(train_s)

print('training time:', time.time()-t1)

training time: 24.751879692077637


In [78]:
test_s = Dataset.load_from_df(testset[sparse_features+y], reader=rdr)
test_s= test_s.build_full_trainset().build_testset()

pred = algo_svd.test(test_s)

print('\nTest results on 80k training data & 20k test data -- rmse: {}, mae: {}'.format(accuracy.rmse(pred), accuracy.mae(pred)))

RMSE: 0.9737
MAE:  0.7629

Test results on 80k training data & 20k test data -- rmse: 0.973745067502651, mae: 0.7629353508722497


* **Reference NN model structure for SVD**


In [6]:
from tensorflow import keras
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras import metrics
from tensorflow.keras.utils import to_categorical

In [213]:
user_in_layer = Input(shape=(df[sparse_features[1]].nunique()), name='user_id')#top half of input layer,+1 to accomodate on-hot encoded vectors
movie_in_layer = Input(shape=(df[sparse_features[0]].nunique()), name= 'movie_id')#bottom half of input layer


#total input moves as concatenated user_id & movie_id one-hot encoded forms

hid_layer_u = Dense(100, name='latent_embed_user_id')(user_in_layer)#latent dimension k=100 for user_id #_Embedding(100, )
hid_layer_m= Dense(100,name='latent_embed_movie_id')(movie_in_layer)#latent dimension k=100 for movie_id

merge_layer = keras.layers.dot([hid_layer_u, hid_layer_m], axes=1)



#predictions = Dense(5, activation='softmax')(merge_layer)#treating problem as multi-class problem
predictions = Dense(1, activation='relu')(merge_layer)

model = Model(inputs=[user_in_layer, movie_in_layer], outputs= predictions)

model.summary()

Model: "model_12"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
user_id (InputLayer)            [(None, 943)]        0                                            
__________________________________________________________________________________________________
movie_id (InputLayer)           [(None, 1682)]       0                                            
__________________________________________________________________________________________________
latent_embed_user_id (Dense)    (None, 100)          94400       user_id[0][0]                    
__________________________________________________________________________________________________
latent_embed_movie_id (Dense)   (None, 100)          168300      movie_id[0][0]                   
___________________________________________________________________________________________

In [89]:
print('feature names:',sparse_features, '\nlabel name:',y)

feature names: ['movie_id', 'user_id'] 
label name: ['rating']


In [208]:
x_train_movie_in = np.asarray(pd.get_dummies(df[sparse_features[0]]))#, num_classes=max(df[sparse_features[0]]))#contains the one-hot encoded movie_id data, shaped (batch_size, max(sm_df[1]))
x_train_user_in = np.asarray(pd.get_dummies(df[sparse_features[1]]))#contains the one-hot encoded user_id data, shaped (batch_size, max(sm_df[0]))


#y_ratings= np.asarray(pd.get_dummies(df[y[0]]))#treating problem as multi-class problem

y_ratings= df[y].values.reshape(-1)
print('shape of x_train_user_in:', x_train_user_in.shape,'\nshape of x_train_movie_in:', x_train_movie_in.shape,
      '\nshape of x_train_user_in:',y_ratings.shape)

shape of x_train_user_in: (100000, 943) 
shape of x_train_movie_in: (100000, 1682) 
shape of x_train_user_in: (100000,)


In [214]:
model.compile(loss='mean_squared_error', optimizer='sgd', metrics= ['mae', 'accuracy'])

history = model.fit([x_train_user_in, x_train_movie_in], y_ratings, verbose=1, epochs=8, batch_size=32,validation_split=0.2)#, callbacks=[checkpoints])

Train on 80000 samples, validate on 20000 samples
Epoch 1/8
80000/80000 [==============================] - 5s 58us/sample - loss: 1.4011 - mae: 0.9791 - accuracy: 0.0616 - val_loss: 1.2513 - val_mae: 0.9407 - val_accuracy: 0.0579
Epoch 2/8
80000/80000 [==============================] - 4s 55us/sample - loss: 1.2713 - mae: 0.9457 - accuracy: 0.0619 - val_loss: 1.2515 - val_mae: 0.9384 - val_accuracy: 0.0579
Epoch 3/8
80000/80000 [==============================] - 5s 60us/sample - loss: 1.2707 - mae: 0.9454 - accuracy: 0.0619 - val_loss: 1.2503 - val_mae: 0.9394 - val_accuracy: 0.0579
Epoch 4/8
80000/80000 [==============================] - 4s 52us/sample - loss: 1.2678 - mae: 0.9443 - accuracy: 0.0619 - val_loss: 1.2457 - val_mae: 0.9399 - val_accuracy: 0.0579
Epoch 5/8
80000/80000 [==============================] - 4s 53us/sample - loss: 1.2417 - mae: 0.9342 - accuracy: 0.0619 - val_loss: 1.1909 - val_mae: 0.9112 - val_accuracy: 0.0579
Epoch 6/8
80000/80000 [===========================

* **SVD Using `deepctr.DeepFM`**
* Following code attempts to follow refrence model & produce a DNN equivalent of SVD using as many DeepCTR's given modules/functionalities as possible.
* Especially layers from `deepctr.models.DeepFM` are tried without disturbing existing `DeepFM( )` function, for resulting similar transformations & producing a SVD model.

In [2]:
import pandas as pd
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

from deepctr.models import DeepFM
from deepctr.inputs import SparseFeat,get_fixlen_feature_names

Using TensorFlow backend.


In [217]:
df.head(4)

,user_id,movie_id,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923


In [5]:
sparse_features = ["movie_id", "user_id"]
y= ['rating']
for feat in sparse_features:
        lbe = LabelEncoder()
        df[feat] = lbe.fit_transform(df[feat])
    # 2.count #unique features for each sparse field
df.head(4)

,user_id,movie_id,rating,timestamp
0,195,241,3,881250949
1,185,301,3,891717742
2,21,376,1,878887116
3,243,50,2,880606923


In [6]:
fixlen_feature_columns = [SparseFeat(feat, df[feat].nunique()) for feat in sparse_features]

linear_feature_columns = fixlen_feature_columns
dnn_feature_columns = fixlen_feature_columns

In [7]:
train, test = train_test_split(df, test_size=0.2)

train_model_input = [train[name].values for name in sparse_features]#includes values from only data[user_id], data[movie_id]
test_model_input = [test[name].values for name in sparse_features]#includes values from only data[user_id], data[movie_id]

print('no. of columns included in train & test:', len(train_model_input))

no. of columns included in train & test: 2


**In the following DeepFM( ) model**
* Embedding size ~ num_factors(surprise SVD)= 100

* The Input layer receives the `OrdinalEncoded` values of `User_id` and `movie_id` which are further one-hot encoded before sparse layers `sparse_emb_movie_id` & `sparse_emb_user_id`.
* The `user_factors` (pu) & `item_factors` (qi) vectors are realised as weights of shape (1682,100) and (943,100) resepectively

In [224]:
print('number of unique features in movie_id:', df[sparse_features[0]].nunique())

number of unique features in movie_id: 1682


In [225]:
print('number of unique features in user_id:', df[sparse_features[1]].nunique())

number of unique features in user_id: 943


In [226]:
model = DeepFM(linear_feature_columns, dnn_feature_columns, embedding_size=100, task='regression')#DNN=Flase
model.compile("adam", "mse", metrics=['mse'], )

model.summary()

Model: "model_13"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
movie_id (InputLayer)           [(None, 1)]          0                                            
__________________________________________________________________________________________________
user_id (InputLayer)            [(None, 1)]          0                                            
__________________________________________________________________________________________________
sparse_emb_movie_id (Embedding) (None, 1, 100)       168200      movie_id[0][0]                   
__________________________________________________________________________________________________
sparse_emb_user_id (Embedding)  (None, 1, 100)       94300       user_id[0][0]                    
___________________________________________________________________________________________

* **Above model summary exhibits some additional/irrelevant layers, which serve no utility in implementing SVD or mimicking structure mentioned before.**

In [25]:
from deepctr.models import svd#svd is the custom module containing code pertaining to SVD's reference NN structure

In [27]:
??svd

* **Now will modify this custom module code using existing deepctr's functionalities (data processing methods & modules).**

In [9]:
from deepctr.inputs import input_from_feature_columns, get_linear_logit,build_input_features,combined_dnn_input
import tensorflow as tf
from deepctr.layers.utils import concat_fun
from deepctr.layers.interaction import FM
from deepctr.layers.core import PredictionLayer

In [236]:
def SVD(feature_columns, embedding_size=100,
        l2_reg_embedding=1e-5, l2_reg_linear=1e-5, l2_reg_dnn=0, init_std=0.0001, seed=1024, bi_dropout=0,
        dnn_dropout=0, act_func='sigmoid', task='binary'):

    features = build_input_features(feature_columns)

    input_layers = list(features.values())
    sparse_embedding_list, _ = input_from_feature_columns(features,feature_columns,
                                                                              embedding_size,
                                                                              l2_reg_embedding,init_std,
                                                                              seed)
    
    fm_input = concat_fun(sparse_embedding_list, axis=1)
    fm_logit = FM()(fm_input)
    
    #hid_layer_1= Dense(num_factors)(input_layers[0])
    #output = PredictionLayer(task)(final_logit)
    model = tf.keras.models.Model(inputs=input_layers, outputs=fm_logit)
    return model

In [237]:
model2 = SVD(linear_feature_columns, embedding_size=100)
model2.summary()

Model: "model_14"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
movie_id (InputLayer)           [(None, 1)]          0                                            
__________________________________________________________________________________________________
user_id (InputLayer)            [(None, 1)]          0                                            
__________________________________________________________________________________________________
sparse_emb_movie_id (Embedding) (None, 1, 100)       168200      movie_id[0][0]                   
__________________________________________________________________________________________________
sparse_emb_user_id (Embedding)  (None, 1, 100)       94300       user_id[0][0]                    
___________________________________________________________________________________________

In [257]:
model2.compile("adam", "mse", metrics=['mse'], )
history = model2.fit(train_model_input, train[y].values, batch_size=64, epochs=8, verbose=2, validation_split=0.2,)

Train on 64000 samples, validate on 16000 samples
Epoch 1/8


c:\users\might\appdata\local\programs\python\python36\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


64000/64000 - 4s - loss: 6.2732 - mse: 6.2476 - val_loss: 1.5095 - val_mse: 1.4566
Epoch 2/8
64000/64000 - 2s - loss: 1.1913 - mse: 1.1293 - val_loss: 1.0942 - val_mse: 1.0256
Epoch 3/8
64000/64000 - 2s - loss: 1.0188 - mse: 0.9470 - val_loss: 1.0355 - val_mse: 0.9608
Epoch 4/8
64000/64000 - 2s - loss: 0.9726 - mse: 0.8961 - val_loss: 1.0217 - val_mse: 0.9437
Epoch 5/8
64000/64000 - 2s - loss: 0.9396 - mse: 0.8602 - val_loss: 1.0089 - val_mse: 0.9283
Epoch 6/8
64000/64000 - 2s - loss: 0.9078 - mse: 0.8260 - val_loss: 0.9962 - val_mse: 0.9132
Epoch 7/8
64000/64000 - 2s - loss: 0.8758 - mse: 0.7918 - val_loss: 0.9913 - val_mse: 0.9062
Epoch 8/8
64000/64000 - 2s - loss: 0.8446 - mse: 0.7583 - val_loss: 0.9842 - val_mse: 0.8970


* Tapping trained equivalent of decomposed matrices (SVD's) `qi` & `pu` at `sparse_emb_movie` & `sparse_emb_user` layers; And estimating ratings for given a given pair of `uid` & `mid`.


In [262]:
#layer2= model2.layers[2].get_weights()[0]#sparse_movie weights Or qi
#print('layer name: ', model2.layers[2].name,'\nshape of weight matrix: ', layer2.shape)

layer name:  sparse_emb_movie_id 
shape of weight matrix:  (1682, 100)


In [263]:
#layer3= model2.layers[3].get_weights()[0]#sparse_user weights Or pu
#print('layer name: ', model2.layers[3].name,'\nshape of weight matrix: ', layer3.shape)

layer name:  sparse_emb_user_id 
shape of weight matrix:  (943, 100)


* Following is the latent output at layer 2 & 3;
* And also the rating result that follows from manual vector product between two latent representations.

In [399]:
from tensorflow.keras import backend as K
t_ui = test_model_input[1][1]
t_mi= test_model_input[0][1]


#outputs the latent reprsentation of movie_id 1033 at layer 2 named:  sparse_emb_user_id 
output_sparse_emb_mov = K.function([model2.layers[0].input],[model2.layers[2].output])
test_output_movie = output_sparse_emb_mov([t_mi])

#outputs the latent reprsentatio of user_id 533 at layer 3 named:  sparse_emb_user_id 
output_sparse_emb_user = K.function([model2.layers[1].input],[model2.layers[3].output])
test_output_user = output_sparse_emb_user([t_ui])

pred_ui_mi = np.dot(test_output_movie[0], test_output_user[0])
print('for user_id: {} & movie_id: {},\npredicted rating from SVD: {}'.format(t_ui, t_mi,pred_ui_mi))

for user_id: 533 & movie_id: 1033,
predicted rating from SVD: 3.3714075088500977


* Comparing above results with usual `model.predict(test_model_input)` i.e., predictions from complete model.

In [293]:
print('Model predictions:', model2.predict([np.array([1033]),np.array([533])]))

Model predictions: [[3.3714075]]


* **Now to further with input transformation from Inputs, latent to final ratings; And understand code underneath `deepctr.layers.interaction.FM`.**


* **Stripping last layer: `FM( )`; Output at layer `concatenate_18`**

* Following is a test method, same as `SVD( )`, only missing link to `FM( )`

In [414]:
model2.layers[4].name

'concatenate_18'

In [18]:
def test_out(feature_columns, embedding_size=100,
        l2_reg_embedding=1e-5, l2_reg_linear=1e-5, l2_reg_dnn=0, init_std=0.0001, seed=1024, bi_dropout=0,
        dnn_dropout=0, act_func='sigmoid', task='binary'):

    features = build_input_features(feature_columns)

    input_layers = list(features.values())
    sparse_embedding_list, _ = input_from_feature_columns(features,feature_columns,
                                                                              embedding_size,
                                                                              l2_reg_embedding,init_std,
                                                                              seed)
    
    fm_input = concat_fun(sparse_embedding_list, axis=1)
    #fm_logit = FM()(fm_input)
    
    #hid_layer_1= Dense(num_factors)(input_layers[0])
    #output = PredictionLayer(task)(final_logit)
    model = tf.keras.models.Model(inputs=input_layers, outputs=fm_input)
    return model

test_output_model = test_out(linear_feature_columns, embedding_size=100)
test_output_model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
movie_id (InputLayer)           [(None, 1)]          0                                            
__________________________________________________________________________________________________
user_id (InputLayer)            [(None, 1)]          0                                            
__________________________________________________________________________________________________
sparse_emb_movie_id (Embedding) (None, 1, 100)       168200      movie_id[0][0]                   
__________________________________________________________________________________________________
sparse_emb_user_id (Embedding)  (None, 1, 100)       94300       user_id[0][0]                    
______________________________________________________________________________________________

In [22]:
test_output= test_output_model.predict([np.array([579]),np.array([171])])

test_output.shape

(1, 2, 100)

In [33]:
print('u_id:', t_ui,'\nm_id:',t_mi)

u_id: 171 
m_id: 579


In [24]:
from tensorflow.keras import backend as K

#outputs the latent reprsentation of movie_id 1033
output_sparse_emb_mov = K.function([test_output_model.layers[0].input],[test_output_model.layers[2].output])
test_output_movie = output_sparse_emb_mov([t_mi])#Same as test_output[0][0]

#outputs the latent reprsentatio of user_id 533
output_sparse_emb_user = K.function([test_output_model.layers[1].input],[test_output_model.layers[3].output])
test_output_user = output_sparse_emb_user([t_ui])#Same as test_output[0][1]

In [25]:
#manual_concat = np.concatenate((test_output_movie[0].reshape(1,-1),test_output_user[0].reshape(1,-1)), axis=0)
#manual_concat.shape#Same as test_output

(2, 100)

* The output here is a `dot` between raw latent embeddings, which are due to be passed into FM layer, wherein the supposition is that the code underlying `deepctr.layers.interaction.FM` is doing operations to more or less produce the same output as dot product.

In [71]:
np.dot(test_output_movie[0], test_output_user[0])# qi*pu

1.2116801e-07

* **Custom `FM( ) layer` code**

* Following computes the feature interaction between `movie_id` & `user_id`

In [16]:
#Input to FM() layer
sparse_embedding_list

[<tf.Tensor 'sparse_emb_movie_id/Identity:0' shape=(None, 1, 100) dtype=float32>,
 <tf.Tensor 'sparse_emb_user_id/Identity:0' shape=(None, 1, 100) dtype=float32>]

In [17]:
fm_input = concat_fun(sparse_embedding_list, axis=1)
fm_input

<tf.Tensor 'concatenate/Identity:0' shape=(None, 2, 100) dtype=float32>

In [372]:
fm_logit= FM(name='fm_layer')(fm_input)
fm_logit

<tf.Tensor 'fm_layer/Identity:0' shape=(None, 1) dtype=float32>

In [43]:
concated_embeds_value= test_output#test_output is obtained from above, for (user,item)
concated_embeds_value.shape

(1, 2, 100)

In [40]:
square_of_sum = tf.square(tf.math.reduce_sum(concated_embeds_value, axis=1, keepdims=True))

sum_of_square = tf.math.reduce_sum(concated_embeds_value * concated_embeds_value, 
                                   axis=1, keepdims=True)

In [66]:
cross_term= square_of_sum - sum_of_square

In [67]:
cross_term= 0.5* tf.math.reduce_sum(cross_term, axis=2, keepdims=False)
cross_term

<tf.Tensor: id=179, shape=(1, 1), dtype=float32, numpy=array([[1.2116803e-07]], dtype=float32)>

* **6 sparse features**

In [78]:
data = pd.read_csv('./movielens_sample.txt')
sparse_features_ext = ["movie_id", "user_id","gender", "age", "occupation", "zip"]
y= ['rating']

In [79]:
for feat in sparse_features_ext:
        lbe = LabelEncoder()
        data[feat] = lbe.fit_transform(data[feat])
        
data.head(3)

,user_id,movie_id,rating,timestamp,title,genres,gender,age,occupation,zip
0,107,12,4,968035345,Ed Wood (1994),Comedy|Drama,0,2,4,35
1,123,169,3,966536874,Patriot Games (1992),Action|Thriller,1,1,4,118
2,12,6,4,976203603,"Bridges of Madison County, The (1995)",Drama|Romance,0,2,13,99


In [80]:
fixlen_feature_columns = [SparseFeat(feat, data[feat].nunique()) for feat in sparse_features_ext]
linear_feature_columns = fixlen_feature_columns

#unique features for each sparse field
linear_feature_columns

[SparseFeat(name='movie_id', dimension=187, use_hash=False, dtype='int32', embedding_name='movie_id', embedding=True),
 SparseFeat(name='user_id', dimension=193, use_hash=False, dtype='int32', embedding_name='user_id', embedding=True),
 SparseFeat(name='gender', dimension=2, use_hash=False, dtype='int32', embedding_name='gender', embedding=True),
 SparseFeat(name='age', dimension=7, use_hash=False, dtype='int32', embedding_name='age', embedding=True),
 SparseFeat(name='occupation', dimension=20, use_hash=False, dtype='int32', embedding_name='occupation', embedding=True),
 SparseFeat(name='zip', dimension=188, use_hash=False, dtype='int32', embedding_name='zip', embedding=True)]

In [81]:
test_output_model_6 = test_out(linear_feature_columns, embedding_size=100)
test_output_model_6.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
movie_id (InputLayer)           [(None, 1)]          0                                            
__________________________________________________________________________________________________
user_id (InputLayer)            [(None, 1)]          0                                            
__________________________________________________________________________________________________
gender (InputLayer)             [(None, 1)]          0                                            
__________________________________________________________________________________________________
age (InputLayer)                [(None, 1)]          0                                            
____________________________________________________________________________________________

In [94]:
train_6, test_6 = train_test_split(data, test_size=0.2)
train_model_input_6 = [train_6[name].values for name in sparse_features_ext]
test_model_input_6 = [test_6[name].values for name in sparse_features_ext]

print('no. of columns included in train & test:', len(train_model_input_6))

no. of columns included in train & test: 6


In [96]:
test_6.head(3)

,user_id,movie_id,rating,timestamp,title,genres,gender,age,occupation,zip
36,16,42,1,975799925,High School High (1996),Comedy,1,1,4,149
34,86,72,5,974435324,Young Frankenstein (1974),Comedy|Horror,1,5,1,69
190,110,169,5,980115327,Patriot Games (1992),Action|Thriller,1,3,7,116


In [108]:
sparse_features_ext

['movie_id', 'user_id', 'gender', 'age', 'occupation', 'zip']

In [114]:
#providing inputs from first index of above dataframe
t_mi = test_model_input_6[0][0]
t_ui =test_model_input_6[1][0]
t_ge =test_model_input_6[2][0]
t_ag= test_model_input_6[3][0]
t_oc= test_model_input_6[4][0]
t_zi= test_model_input_6[5][0]

In [109]:
len(test_output_model_6.layers)

13

In [115]:
from tensorflow.keras import backend as K

#outputs the latent reprsentation of movie_id 1033
output_sparse_emb_mov_6 = K.function([test_output_model_6.layers[0].input],[test_output_model_6.layers[6].output])
test_output_movie_6 = output_sparse_emb_mov_6([t_mi])

#outputs the latent reprsentatio of user_id 533
output_sparse_emb_user_6 = K.function([test_output_model_6.layers[1].input],[test_output_model_6.layers[7].output])
test_output_user_6 = output_sparse_emb_user_6([t_ui])

output_sparse_emb_gender_6 = K.function([test_output_model_6.layers[2].input],[test_output_model_6.layers[8].output])
test_output_gender_6 = output_sparse_emb_gender_6([t_ge])

output_sparse_emb_age_6 = K.function([test_output_model_6.layers[3].input],[test_output_model_6.layers[9].output])
test_output_age_6 = output_sparse_emb_age_6([t_ag])

output_sparse_emb_oc_6 = K.function([test_output_model_6.layers[4].input],[test_output_model_6.layers[10].output])
test_output_oc_6 = output_sparse_emb_oc_6([t_oc])

output_sparse_emb_zip_6 = K.function([test_output_model_6.layers[5].input],[test_output_model_6.layers[11].output])
test_output_zip_6 = output_sparse_emb_zip_6([t_zi])


* Following computes feature interaction between above 6 latent embeddings through FM( ) layer operations, same as before whihch results rating values.

In [103]:
test_output_6= test_output_model_6.predict(test_model_input_6, batch_size=32)
#test_output_6= test_output_model_6.predict([test_model_input_6[0][1],test_model_input_6[1][1], test_model_input_6[2][1],test_model_input_6[3][1],test_model_input_6[4][1],test_model_input_6[5][1]])
test_output_6.shape

(40, 6, 100)

In [125]:
concated_embeds_value= test_output_6[0].reshape(1,6,100)#test_output is same as fm_logit, for (user,item)
concated_embeds_value.shape

(1, 6, 100)

In [126]:
square_of_sum = tf.square(tf.math.reduce_sum(concated_embeds_value, axis=1, keepdims=True))

sum_of_square = tf.math.reduce_sum(concated_embeds_value * concated_embeds_value, 
                                   axis=1, keepdims=True)

In [127]:
cross_term= square_of_sum - sum_of_square

cross_term= 0.5* tf.math.reduce_sum(cross_term, axis=2, keepdims=False)
cross_term

<tf.Tensor: id=460, shape=(1, 1), dtype=float32, numpy=array([[1.2463119e-07]], dtype=float32)>

In [120]:
np.dot(test_output_movie_6[0],test_output_user_6[0])
#,test_output_gender_6[0],test_output_age_6[0],
       #test_output_oc_6[0],test_output_zip_6[0])

-2.3915721e-08